In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install imutils

You should consider upgrading via the 'd:\anaconda\python.exe -m pip install --upgrade pip' command.


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [4]:
INIT_LR = 1e-4
EPOCHS = 8
BS = 32

In [5]:
1. ## print("[INFO] loading images...")
imagePaths = list(paths.list_images(('./input/dataset/')))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
    
# extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
# load the input image (224x224) and preprocess it
    image = load_img(imagePath,target_size=[512,512])
    image = img_to_array(image)
    image = preprocess_input(image)
# update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(512, 512, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [8]:
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

[INFO] compiling model...
[INFO] training head...
Epoch 1/8
34/34 [==============================] - 80s 2s/step - loss: 0.5105 - accuracy: 0.7706 - val_loss: 0.1886 - val_accuracy: 0.9420
Epoch 2/8
34/34 [==============================] - 79s 2s/step - loss: 0.2197 - accuracy: 0.9148 - val_loss: 0.1457 - val_accuracy: 0.9384
Epoch 3/8
34/34 [==============================] - 82s 2s/step - loss: 0.1699 - accuracy: 0.9401 - val_loss: 0.0819 - val_accuracy: 0.9819
Epoch 4/8
34/34 [==============================] - 80s 2s/step - loss: 0.1316 - accuracy: 0.9494 - val_loss: 0.0861 - val_accuracy: 0.9746
Epoch 5/8
34/34 [==============================] - 83s 2s/step - loss: 0.0920 - accuracy: 0.9700 - val_loss: 0.0780 - val_accuracy: 0.9746
Epoch 6/8
34/34 [==============================] - 82s 2s/step - loss: 0.0932 - accuracy: 0.9691 - val_loss: 0.0779 - val_accuracy: 0.9710
Epoch 7/8
34/34 [==============================] - 83s 2s/step - loss: 0.0753 - accuracy: 0.9738 - val_loss: 0.0567 

In [9]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))
# serialize the model to disk
print("[INFO] saving mask detector model...")
tf.saved_model.save(obj=model,export_dir='/models/mask_detector_model')

[INFO] evaluating network...
              precision    recall  f1-score   support

   with_mask       1.00      0.96      0.98       138
without_mask       0.96      1.00      0.98       138

    accuracy                           0.98       276
   macro avg       0.98      0.98      0.98       276
weighted avg       0.98      0.98      0.98       276

[INFO] saving mask detector model...
